# 爬虫请求库之Urllib

在Python 2中，有urllib和urllib2两个库来实现请求的发送。而在Python 3中，已经不存在urllib2这个库了，统一为urllib，官方文档点击[这里](https://docs.python.org/3/library/urllib.html)。

首先，了解一下urllib库，它是Python内置的HTTP请求库，也就是说不需要额外安装即可使用。它包含如下4个模块：
+ urllib.request：它是最基本的HTTP请求模块，可以用来模拟发送请求。就像在浏览器里输入网址然后回车一样，只需要给库方法传入URL以及额外的参数，就可以模拟实现这个过程了。
+ urllib.parse：一个工具模块，提供了许多URL处理方法，比如拆分、解析、合并等。
+ urllib.error：异常处理模块，如果出现请求错误，我们可以捕获这些异常，然后进行重试或其他操作以保证程序不会意外终止。
+ urllib.robotparser：主要是用来识别网站的robots.txt文件，然后判断哪些网站可以爬，哪些网站不可以爬，它其实用得比较少。

# 1. urllib.request

urllib.request模块提供了最基本的构造HTTP请求的方法，利用它可以模拟浏览器的一个请求发起过程，同时它还带有处理授权验证（authenticaton）、重定向（redirection)、浏览器Cookies以及其他内容。

## 1.1 urlopen

如果想给链接传递一些参数，该怎么实现呢？首先看一下urlopen()函数的API，[官方文档](https://docs.python.org/3/library/urllib.request.html#urllib.request.urlopen)。

```python
urllib.request.urlopen(url, data=None, [timeout, ]*, cafile=None, capath=None, cadefault=False, context=None)¶
```

+ **url** 打开URL，可以接受一个字符串的URL，或者一个Request对象；
+ **data** 该参数是可选的。如果要添加该参数，并且如果它是字节流编码格式的内容，即bytes类型，则需要通过bytes()方法转化。另外，如果传递了这个参数，则它的请求方式就不再是GET方式，而是POST方式。
+ **timeout** 该参数用于设置超时时间，单位为秒，意思就是如果请求超出了设置的这个时间，还没有得到响应，就会抛出异常。如果不指定该参数，就会使用全局默认时间。它支持HTTP、HTTPS、FTP请求。

context参数，它必须是ssl.SSLContext类型，用来指定SSL设置。此外，cafile和capath这两个参数分别指定CA证书和它的路径，这个在请求HTTPS链接时会有用。cadefault参数现在已经弃用了，其默认值为False。

我们重点看一下前三个参数的使用：

In [3]:
from urllib import request

html=request.urlopen(url='https://timgsa.baidu.com/timg?image&quality=80&size=b9999_10000&sec=1546854620093&di=eb9e0420a9926dd85b219040ba8f50e2&imgtype=0&src=http%3A%2F%2Fpic31.nipic.com%2F20130702%2F7447430_091916999000_2.jpg')#get
img=html.read()
with open('meizi.jpg','wb') as f:
    f.write(img)

In [ ]:
from urllib import request

html=request.urlopen(url='http://www.baidu.com',data=None)#get
print(html.read().decode('utf-8'))

In [ ]:
from urllib import request
from urllib import parse

data = {
    'hello':'人工智能',
    'wd':'python'
}
# print(parse.urlencode(data))

data = bytes(parse.urlencode(data),encoding='utf-8')
# print(data)
response = request.urlopen('http://httpbin.org/post',data=data)
print(response.read().decode('utf-8'))

下面看看第三个参数，关于timeout,超时的设置：

In [ ]:
from urllib import request

response = request.urlopen('http://www.baidu.com',timeout=0.01)
print(response.read().decode('utf-8'))

In [ ]:
from urllib import request,error
import socket

try:
    response = request.urlopen('https://www.baidu.com',timeout=0.01)
    print('成功请求')
except error.URLError as e:
#     if isinstance(e.reason, socket.timeout):
    print('Time Out')

## 1.2 响应

首先来看看响应的类型：

In [40]:
from urllib import request


response = request.urlopen('http://www.douban.com')
print(type(response))
print(response.reason)#状态码
# print(response.getheaders())#响应头
# print(response.getheader('Date'))
# print(response.read().decode('utf-8'))#响应体

<class 'http.client.HTTPResponse'>
OK


可以发现，它是一个HTTPResposne类型的对象。它主要包含read()、readinto()、getheader(name)、getheaders()、fileno()等方法，以及msg、version、status、reason、debuglevel、closed等属性。查看更所属性点击[这里](https://docs.python.org/3/library/http.client.html#httpresponse-objects)。

这样我们可以查看响应的状态、响应头等信息,使我们判断响应是否成功的重要标志：

## 1.3 Request

我们知道利用urlopen()方法可以实现最基本请求的发起，但这几个简单的参数并不足以构建一个完整的请求。如果请求中需要加入Headers等信息，就可以利用更强大的Request类来构建。详情点击[这里](https://docs.python.org/3/library/urllib.request.html#urllib.request.Request)。

```
urllib.request.Request(url, data=None, headers={}, origin_req_host=None, unverifiable=False, method=None)
```

+ 第一个参数url用于请求URL，这是必传参数，其他都是可选参数。
+ 第二个参数data如果要传，必须传bytes（字节流）类型的。如果它是字典，可以先用urllib.parse模块里的urlencode()编码。
+ 第三个参数headers是一个字典，它就是请求头，我们可以在构造请求时通过headers参数直接构造，也可以通过调用请求实例的add_header()方法添加。
+ 第四个参数origin_req_host指的是请求方的host名称或者IP地址。
+ 第五个参数unverifiable表示这个请求是否是无法验证的，默认是False，意思就是说用户没有足够权限来选择接收这个请求的结果。例如，我们请求一个HTML文档中的图片，但是我们没有自动抓取图像的权限，这时unverifiable的值就是True`。
+ 第六个参数method是一个字符串，用来指示请求使用的方法，比如GET、POST和PUT等。

首先，我们用实例来感受一下Request的用法

下面我们传入多个参数构建请求来看一下：

In [27]:
from urllib import request

headers={
    'Referer':'www.mmjpg.com/mm/1587',
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36'
}
request1 = request.Request('http://fm.shiyunjj.com/2019/1587/2igo.jpg',headers=headers)
response = request.urlopen(request1)
with open('aaa.jpg','wb') as f:
    f.write(response.read())

In [29]:
from urllib import request

from urllib import parse

headers={
    'Referer':'www.mmjpg.com/mm/1587',
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36'
}
d = {
    'wd':'python'
}
data = bytes(parse.urlencode(d),encoding='utf-8')
request1 = request.Request('http://httpbin.org/post',headers=headers,data=data,method='POST')
response = request.urlopen(request1)
print(response.read().decode('utf-8'))

{
  "args": {}, 
  "data": "", 
  "files": {}, 
  "form": {
    "wd": "python"
  }, 
  "headers": {
    "Accept-Encoding": "identity", 
    "Connection": "close", 
    "Content-Length": "9", 
    "Content-Type": "application/x-www-form-urlencoded", 
    "Host": "httpbin.org", 
    "Referer": "www.mmjpg.com/mm/1587", 
    "User-Agent": "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36"
  }, 
  "json": null, 
  "origin": "183.15.178.143", 
  "url": "http://httpbin.org/post"
}



另外，headers也可以用add_header()方法来添加：

In [30]:
from urllib import request, parse
 
url = 'http://httpbin.org/post'
dict = {
    'name': 'Germey'
}
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.186 Safari/537.36',
    'Host': 'httpbin.org/post'
}
data = bytes(parse.urlencode(dict), encoding='utf8')
req = request.Request(url=url, data=data,  method='POST',headers = headers)
req.add_header('User-Agent','nidaye')
req.add_header('Accept-Language','zh-CN,zh;q=0.9')
response = request.urlopen(req)
print(response.read().decode('utf-8'))

{
  "args": {}, 
  "data": "", 
  "files": {}, 
  "form": {
    "name": "Germey"
  }, 
  "headers": {
    "Accept-Encoding": "identity", 
    "Accept-Language": "zh-CN,zh;q=0.9", 
    "Connection": "close", 
    "Content-Length": "11", 
    "Content-Type": "application/x-www-form-urlencoded", 
    "Host": "httpbin.org/post", 
    "User-Agent": "nidaye"
  }, 
  "json": null, 
  "origin": "183.15.178.143", 
  "url": "http://httpbin.org/post/post"
}



## 1.4 Handler

对于一些更高级的操作（比如Cookies处理、代理设置等），就需要更强大的工具Handler登场了。简而言之，我们可以把它理解为各种处理器，有专门处理登录验证的，有处理Cookies的，有处理代理设置的，相当于额外的处理工具。利用它们，我们几乎可以做到HTTP请求中所有的事情。

### 1.4.1 代理([ProxyHandler](https://docs.python.org/3/library/urllib.request.html#proxyhandler-objects))

在做爬虫时，免不了要使用代理，针对urllib.request模块可以这样做：

In [31]:
from urllib import request

req = request.Request('http://httpbin.org/get')
# print(type(req))
response = request.urlopen(req)
print(response.read().decode('utf-8'))

{
  "args": {}, 
  "headers": {
    "Accept-Encoding": "identity", 
    "Connection": "close", 
    "Host": "httpbin.org", 
    "User-Agent": "Python-urllib/3.7"
  }, 
  "origin": "183.15.178.143", 
  "url": "http://httpbin.org/get"
}



In [32]:
from urllib.error import URLError
from urllib.request import ProxyHandler, build_opener
 
proxy_handler = ProxyHandler({
    'https':'https://111.177.177.248:9999'
})
opener = build_opener(proxy_handler)
try:
    response = opener.open('https://httpbin.org/get')
    print(response.read().decode('utf-8'))
except URLError as e:
    print(e.reason)

{
  "args": {}, 
  "headers": {
    "Accept-Encoding": "identity", 
    "Connection": "close", 
    "Host": "httpbin.org", 
    "User-Agent": "Python-urllib/3.7"
  }, 
  "origin": "111.177.177.248", 
  "url": "https://httpbin.org/get"
}



这里主要获得[西刺](http://www.xicidaili.com/)的两个免费代理。这里使用了ProxyHandler，其中一个参数是字典，键名是协议（http或https），键值是代理链接，可以添加多个代理。然后，利用这个Handler及build_opener()方法来使用代理。

### 1.4.2 Cookies([HTTPCookieProcessor](https://docs.python.org/3/library/urllib.request.html#httpcookieprocessor-objects))

在urllib.request模块中,Cookies模块的处理就需要用到Handle了，先看看怎么将网站的Cookies获取下来:

In [ ]:
from http import cookiejar
from urllib import request

cookies = cookiejar.CookieJar()
handler = request.HTTPCookieProcessor(cookies)
opener = request.build_opener(handler)
response = opener.open('http://www.baidu.com')
for item in cookies:
    print(item.name + '=' + item.value)

在实际的应用中，我们需要将cookies获取下来，便于长时间维持账号登录，下面看看输出成文件的例子：

In [ ]:
from http import cookiejar
from urllib import request

filename = 'cookies.txt'
cookie = cookiejar.MozillaCookieJar(filename)
handler = request.HTTPCookieProcessor(cookie)
opener = request.build_opener(handler)
response = opener.open('http://www.baidu.com')
cookie.save(ignore_discard = True,ignore_expires=True)

上面用到的是MozillaCookieJar，将Cookies保存成Mozilla型浏览器的Cookies格式；下面的LWPCookieJar，可以保存成libwww-perl(LWP)格式的Cookies文件。

In [ ]:
from http import cookiejar
from urllib import request

filename = 'cookies.txt'
cookie = cookiejar.LWPCookieJar(filename)
handler = request.HTTPCookieProcessor(cookie)
opener = request.build_opener(handler)
response = opener.open('http://www.baidu.com')
cookie.save(ignore_discard = True,ignore_expires=True)

两者生成的格式差异比较大，所以通过什么格式生成的，就通过同样的方法去读取：

In [ ]:
from http import cookiejar
from urllib import request

cookie = cookiejar.LWPCookieJar()
cookie.load('cookies.txt',ignore_discard = True,ignore_expires=True)
handler = request.HTTPCookieProcessor(cookie)
opener = request.build_opener(handler)
response = opener.open('http://www.baidu.com')
print(response.read().decode('utf-8'))

通过上面的方法，可以实现绝大多数请求功能的设置，关于urllib.request模块的更加强大的功能可以参考[官方文档](https://docs.python.org/3/library/urllib.request.html#)。

# 2. urllib.error

urllib的error模块定义了由request模块产生的异常。如果出现了问题，request模块便会抛出error模块中定义的异常。详细情况见[官方文档](https://docs.python.org/3/library/urllib.error.html#module-urllib.error)。

urllib.error模块主要包含两个错误类，URLError类和HTTPError类。

URLError类来自urllib库的error模块，它继承自OSError类，是error异常模块的基类，由request模块生的异常都可以通过捕获这个类来处理。它具有一个属性reason，即返回错误的原因。

HTTPError类它是URLError的子类，专门用来处理HTTP请求错误，比如认证请求失败等。它有如下3个属性：
+ code：返回HTTP状态码，比如404表示网页不存在，500表示服务器内部错误等。
+ reason：同父类一样，用于返回错误的原因。
+ headers：返回请求头。

因为URLError是HTTPError的父类，所以可以先选择捕获子类的错误，再去捕获父类的错误，所以上述代码更好的写法如下：

这样就可以做到先捕获HTTPError，获取它的错误状态码、原因、headers等信息。如果不是HTTPError异常，就会捕获URLError异常，输出错误原因。最后，用else来处理正常的逻辑。这是一个较好的异常处理写法。有时候，reason属性返回的不一定是字符串，也可能是一个对象

In [ ]:
import socket
from urllib import request, error
 
try:
    response = request.urlopen('http://www.douban.com', timeout=0.1)
except error.HTTPError as e:
    print(e.reason, e.code, e.headers, sep='\n')
except error.URLError as e:
    print(type(e.reason))
    if isinstance(e.reason, socket.timeout):
        print('TIME OUT')
else:
    print('Request Successfully')

# 3. urllib.parse

urllib库提供了parse这个模块，它定义了处理URL的标准接口，例如实现URL各部分的抽取、合并以及链接转换。它支持如下协议的URL处理：file、ftp、gopher、hdl、http、https、imap、mailto、 mms、news、nntp、prospero、rsync、rtsp、rtspu、sftp、 sip、sips、snews、svn、svn+ssh、telnet和wais。详细的介绍点击[这里](https://docs.python.org/3/library/urllib.parse.html)。

## 3.1 urlparse

该方法可以实现URL的识别和分段，这里先用一个实例来看一下:

In [52]:
from urllib.parse import urlparse
 
result = urlparse('http://www.baidu.com/index.html;user?id=5#comment')
print(type(result), result)

<class 'urllib.parse.ParseResult'> ParseResult(scheme='http', netloc='www.baidu.com', path='/index.html', params='user', query='id=5', fragment='comment')


观察一下该实例的URL：

```
urllib.parse.urlparse(urlstring, scheme='', allow_fragments=True)
```

In [ ]:
from urllib.parse import urlparse
 
result = urlparse('www.baidu.com/index.html;user?id=5#comment', scheme='https')
print(result)

可见，scheme参数只有在URL中不包含scheme信息时才生效。如果URL中有scheme信息，就会返回解析出的scheme。

In [ ]:
from urllib.parse import urlparse
 
result = urlparse('http://www.baidu.com/index.html;user?id=5#comment', scheme='https')
print(result)

allow_fragments：即是否忽略fragment。如果它被设置为False，fragment部分就会被忽略，它会被解析为path、parameters或者query的一部分，而fragment部分为空。下面我们用实例来看一下：

In [ ]:
from urllib.parse import urlparse
 
result = urlparse('http://www.baidu.com/index.html;user?id=5#comment', allow_fragments=False)
print(result)

返回结果ParseResult实际上是一个元组，我们可以用索引顺序来获取，也可以用属性名获取。

In [ ]:
print(result[1])
print(result.netloc)

## 3.2 urlunparse

有了urlparse()，相应地就有了它的对立方法urlunparse()。它接受的参数是一个可迭代对象，但是它的长度必须是6，否则会抛出参数数量不足或者过多的问题。先用一个实例看一下：

In [53]:
from urllib.parse import urlunparse
 
data = ['http', 'www.baidu.com', 'index.html', 'user', 'a=6', 'comment']
print(urlunparse(data))

http://www.baidu.com/index.html;user?a=6#comment


## 3.3 urljoin

合成链接的一个方法，那就是urljoin()方法。我们可以提供一个base_url（基础链接）作为第一个参数，将新的链接作为第二个参数，该方法会分析base_url的scheme、netloc和path这3个内容并对新链接缺失的部分进行补充，最后返回结果。

In [ ]:
from urllib.parse import urljoin
 
print(urljoin('http://www.baidu.com', 'FAQ.html'))
print(urljoin('FAQ.html', 'http://www.douban.com'))
print(urljoin('https://www.baidu.com/about.html', 'http://douban.com/FAQ.html'))
print(urljoin('http://www.baidu.com/about.html?question=2', 'https://douban.com/FAQ.html'))
print(urljoin('http://www.baidu.com/faq.html?wd=abc', 'https://doubna.com'))
# print(urljoin('http://www.baidu.com', '?category=2#comment'))
# print(urljoin('www.baidu.com', '?category=2#comment'))
# print(urljoin('www.baidu.com#comment', '?category=2'))

In [ ]:
help(urljoin)

通过urljoin()方法，我们可以轻松实现链接的解析、拼合与生成

## 3.4 urlencode

这里我们再介绍一个常用的方法——urlencode()，它在构造GET请求参数的时候非常有用，示例如下：

In [84]:
from urllib.parse import urlencode
city ='深圳'
params = {
    'city':city,
    'needAddtionalResult': 'false'
}

base_url='https://www.lagou.com/jobs/positionAjax.json?'
s = urlencode(params)
print(base_url+s)

https://www.lagou.com/jobs/positionAjax.json?city=%E6%B7%B1%E5%9C%B3&needAddtionalResult=false


可以看到，参数就成功地由字典类型转化为GET请求参数了。这个方法非常常用。有时为了更加方便地构造参数，我们会事先用字典来表示。要转化为URL的参数时，只需要调用该方法即可。

## 3.5 quote

该方法可以将内容转化为URL编码的格式。URL中带有中文参数时，有时可能会导致乱码的问题，此时用这个方法可以将中文字符转化为URL编码，示例如下：

In [76]:
from urllib.parse import quote
 
keyword = '人工智能'
print(quote(keyword))
url = 'https://www.baidu.com/s?wd=' + quote(keyword)
print(url)

%E4%BA%BA%E5%B7%A5%E6%99%BA%E8%83%BD
https://www.baidu.com/s?wd=%E4%BA%BA%E5%B7%A5%E6%99%BA%E8%83%BD


### 3.6 unquote

In [1]:
from urllib.parse import unquote
unquote('https://www.baidu.com/s?wd=%E4%BA%BA%E5%B7%A5%E6%99%BA%E8%83%BD')

'https://www.baidu.com/s?wd=人工智能'

# 实战练习

在拉钩网上搜索“数据分析”岗位，获取深圳地区的数据分析岗位的相关信息：
+ Ajax分析
+ 使用urllib请求，获取响应信息；
+ 使用json库，获取数据
+ 使用MongoDB保存数据